In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/stockmarket-sentiment-dataset/stock_data.csv', encoding='utf-8')

In [3]:
df

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1
...,...,...
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1


In [4]:
df.columns

Index(['Text', 'Sentiment'], dtype='object')

In [5]:
df.columns=['review','sentiment']

In [6]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [7]:
# Define the custom labels
positive = 'Positive'
negative = 'Negative'

# Replace values in the 'sentiment' column with custom labels
df['sentiment'] = df['sentiment'].replace({1: positive, -1: negative})

In [8]:
df.shape

(5791, 2)

In [9]:
df

,review,sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,Positive
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,Positive
2,user I'd be afraid to short AMZN - they are lo...,Positive
3,MNTA Over 12.00,Positive
4,OI Over 21.37,Positive
...,...,...
5786,Industry body CII said #discoms are likely to ...,Negative
5787,"#Gold prices slip below Rs 46,000 as #investor...",Negative
5788,Workers at Bajaj Auto have agreed to a 10% wag...,Positive
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",Positive


In [10]:
df.isnull().any()

review       False
sentiment    False
dtype: bool

In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
df.describe()

,review,sentiment
count,5791,5791
unique,5791,2
top,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,Positive
freq,1,3685


In [13]:
df['sentiment'].value_counts()

sentiment
Positive    3685
Negative    2106
Name: count, dtype: int64

In [14]:
df.shape

(5791, 2)

# **Text Normalization**

**Tokenization**

In [15]:
# Import data science libraries
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

In [16]:
!pip install toktok

  Preparing metadata (setup.py) ... - done
  Created wheel for toktok: filename=toktok-0.0.2-py3-none-any.whl size=4239 sha256=9e27363448cef22f3e8a5a9c1ecd9bcc8db255684681d0ea849451f56e9f6b31
  Stored in directory: /root/.cache/pip/wheels/ba/af/b7/4d2a15cdf2dc37ac44fb90376822fd55b6e41c536ec5d9e4b6
Successfully built toktok


In [17]:
import spacy
import re
import string
import unicodedata
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix
from bs4 import BeautifulSoup

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus.reader.tagged import ToktokTokenizer
tokenizers = ToktokTokenizer()
stopwords = nltk.corpus.stopwords.words('english')

In [20]:
def noiseremovel_text(text):
  soup = BeautifulSoup(text, "html.parser")
  text = soup.get_text()
  text = re.sub('\[[^]]*\]', '',text)
  return text

In [21]:
df['review'] = df['review'].apply(noiseremovel_text)

/tmp/ipykernel_18/1987957958.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [22]:
df.head()

,review,sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,Positive
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,Positive
2,user I'd be afraid to short AMZN - they are lo...,Positive
3,MNTA Over 12.00,Positive
4,OI Over 21.37,Positive


# **Stemming**

In [23]:
def stemmer(text):
  ps = nltk.porter.PorterStemmer()
  text = '.'.join([ps.stem(word) for word in text.split()])
  return text

In [24]:
df['review'] = df['review'].apply(stemmer)

In [25]:
df.head()

,review,sentiment
0,kicker.on.my.watchlist.xide.tit.soq.pnk.cpw.bp...,Positive
1,user:.aap.movie..55%.return.for.the.fea/ge.ind...,Positive
2,user.i'd.be.afraid.to.short.amzn.-.they.are.lo...,Positive
3,mnta.over.12.00,Positive
4,OI.over.21.37,Positive


In [26]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer

# Assuming you have a DataFrame named 'df' with a column 'review'

# Download the necessary data
nltk.download('punkt')
nltk.download('stopwords')

# Function to remove stopwords from text
def remove_stopwords(text, stop_words=None, is_lower_case=False):
    # Create a ToktokTokenizer instance
    tokenizers = ToktokTokenizer()

    # Use a regular expression to split the text into words while preserving consecutive full-stop signs
    words = re.findall(r'\w+|\.\.+', text)

    # Remove stopwords
    if stop_words is None:
        stop_words = set()
    else:
        stop_words = set(stop_words)

    if is_lower_case:
        # Remove stopwords without converting tokens to lowercase
        filtokens = [i for i in words if i not in stop_words]
    else:
        # Remove stopwords after converting tokens to lowercase
        filtokens = [i for i in words if i.lower() not in stop_words]

    # Join the filtered words back into a sentence
    filtered_text = ' '.join(filtokens)
    return filtered_text

# Now, apply the remove_stopwords function to the 'review' column of the DataFrame

# Get the English stopwords
stop_wr = set(stopwords.words('english'))

# Apply the remove_stopwords function to the 'review' column of the DataFrame
df['review'] = df['review'].apply(remove_stopwords, stop_words=stop_wr)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
df.head()

,review,sentiment
0,kicker watchlist xide tit soq pnk cpw bpz AJ t...,Positive
1,user aap movie .. 55 return fea ge indic 15 tr...,Positive
2,user afraid short amzn look like near monopoli...,Positive
3,mnta 12 00,Positive
4,OI 21 37,Positive


# **Traning Test Split**

In [28]:
X = df.review[:3000]

In [29]:
y = df.review[3000:]

# **Bag of Words**

In [30]:
cv = CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
cv_train = cv.fit_transform(X)
cv_test = cv.fit_transform(y)
print('Bow_cv_train',cv_train.shape)
print('Bow_cv_test',cv_test.shape)

Bow_cv_train (3000, 43267)
Bow_cv_test (2791, 44484)


# **TE_IDF**

In [31]:
tf = TfidfVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
tf_train = tf.fit_transform(X)
tf_test = tf.fit_transform(y)
print('Bow_tf_train',tf_train.shape)
print('Bow_tf_test',tf_test.shape)

Bow_tf_train (3000, 43267)
Bow_tf_test (2791, 44484)


# **Label_Encoding**

In [32]:
# Create a LabelBinarizer instance
label_binarizer = LabelBinarizer()

# Fit and transform the 'sentiment' column
sentiment_encoded = label_binarizer.fit_transform(df['sentiment'])
print(sentiment_encoded.shape)

(5791, 1)


In [33]:
sentiment_encoded

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
# Create a CountVectorizer instance
vectorizer = CountVectorizer()

# Convert the text data to bag-of-words features
X_bow = vectorizer.fit_transform(df['review'])

# Split the data into features (X) and target (y)
X_train, X_test, y_train, y_test = train_test_split(X_bow, df['sentiment'], test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
logistic = LogisticRegression(penalty='l2', max_iter=500, C=1, random_state=42)

# Fitting the model on the training data
lr_bow = logistic.fit(X_train, y_train)

# Make predictions on the test data
y_pred = lr_bow.predict(X_test)

# Calculate and print accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.8006902502157032
Classification Report:
              precision    recall  f1-score   support

    Negative       0.75      0.69      0.72       427
    Positive       0.83      0.87      0.85       732

    accuracy                           0.80      1159
   macro avg       0.79      0.78      0.78      1159
weighted avg       0.80      0.80      0.80      1159



In [35]:
# Make predictions on new text data
new_text = ["Volatility's grip tightens, stocks plunge in a sea of red."]
new_text_bow = vectorizer.transform(new_text)
y_pred = lr_bow.predict(new_text_bow)

print("Predicted Sentiment:", y_pred[0])

Predicted Sentiment: Negative
